In [1]:
import pandas as pd
import numpy as np
import hebrew_tokenizer as tokenizer
import re
# from yap_api import YapApi
from collections import defaultdict

C:\Users\olgaf\anaconda3\lib\site-packages\hebrew_tokenizer\__init__.py:5: FutureWarning: Possible nested set at position 20
  from .api import tokenize


In [3]:
class Hebrew_preprocessing:
    def __init__ (self, additional_replacements = []):
     
        #list of tuples, where tuples are paterns to be replaced
        base_replacements = [
               (r'(https.*$ )', ' '),           # remove links. here regex is used instead of Beautiful soup, because Beutiful soup doesn't work with Hebrew.
               (r'href="(.*?)"', ' '),
               (r'\\n?(https.*)\\n', ' '),
               (r'(https.*)\\n', ' '),          # remove html tags
               (r'(\&nbsp;)', ' '),             
               (r'(\\&quot;)', ' '),
               (r'<.*?>', ' '),                 # remove string in <> 
               (r'\n',' '),                     # remove newline
               (r'(\\r)',' '),
               (r'[\:;&#=\{\}\(\)]',' '),       # remove special chatacters. again, [a-z0-9] won't work because of Hebrew
               (r'(\/+)', ' '),
               (r'(\\+)', ' '),
               (r'(\++)', ' '),
               (r' - ', r' '),
               (r'(\s+)', r' ')                 # remove multiple spaces, created by the above cleaning 
              ]
        self.replacements = base_replacements + additional_replacements
        
    def regex_replacement(self, string:str, custom_replacements = []) -> str:             
        for pat,repl in self.replacements + custom_replacements:
            string = re.sub(pat, repl, string)
        return string        
    
    def get_groups_tokenized(self, text:str)-> dict():
        tokens = tokenizer.tokenize(text)
        groups = defaultdict(list)
        for grp, token, token_num, (start_index, end_index) in tokens:
            group,tok = str(grp), str(token)
            if group == 'Groups.HEBREW':
                groups['tokens'].append(token)
            if group == 'Groups.PUNCTUATION':
                groups['punct'].append(token)
            if group == 'Groups.NUMBER':
                groups['numbers'].append(token)
        return groups  
    
    def tokenize(self, text:str)-> list():
        return self.get_groups_tokenized(text)['tokens']
    
    def punctuation(self, text:str)-> list():
        return self.get_groups_tokenized(text)['punct']
    
    def punctuation_count(self, text:str)-> list():
        return len(self.get_groups_tokenized(text)['punct'])
    
    def numbers(self, text:str)-> list():
        return self.get_groups_tokenized(text)['numbers']
    
    def numbers_count(self, text:str)-> list():
        return len(self.get_groups_tokenized(text)['numbers'])
    
    """
    Set up Hebrew Lematizer before the function. It need Go pre-installed. 
    Documentation: https://nlp.biu.ac.il/~rtsarfaty/onlp/hebrew/documentation
    IP of YAP server: ip = '127.0.0.1:8000'
    yap = YapApi()
    """
    def lemmatize(self, list_tokens:list) -> str:
        text = ' '.join(list_tokens)
        tokenized_text, segmented_text, lemmas, dep_tree, md_lattice, ma_lattice = yap.run(text, ip)
        return lemmas
    """
    Hebrew stopwords are loaded from "he_stopwords.txt" file. 
    """
    def count_stopwords(self,text:str, stopwords) -> int:
        tokens = text#.split(' ')   
        stopw = []
        count = 0
        for w in stopwords:
            if w in tokens:
                for i in range(len(tokens)): 
                    if (w == tokens[i]): 
                        count = count + 1
                stopw.append(w)
        return count
        
    def remove_stopwords(self,text:str, stopwords) -> str:
        tokens = text#.split(' ') 
        stopw = []
        for w in stopwords:
            while w in tokens:
                tokens.remove(w)
        return text

    def save_csv(self, dataFrame, name:str):
        dataFrame.to_csv(name+'.csv',index=False)

In [4]:
file = pd.read_excel('All_Forum_Posts_with_COVID_new.xlsx')
file.head()

,Time,Forum,name,title,post,repsonses,informative,code1_i,code2_i,code3_i,...,place,filter_$,xxx,ei,word,words,ID,ID_nu,title_words,posts_words
0,0,1,בחור מבולבל,משיכה לחיתולים ומוצרי תינוקות ומזוכיזם,NaN,5,0,0,0,0,...,0,0,1,0,6,6,76,0,5,1
1,0,1,scarlette,"בואו נתאחד כדי לתת סיכוי להצעת החוק של ח""כ גילאון",להעלות את קצבאות הנכות לגובה שכר המינימום.<br ...,2,0,0,0,0,...,2,0,1,0,143,143,77,0,10,133
2,0,1,Tictac,אם אתם לוקים בחרדה חברתית,אני מבקש מכם לענות על שאלון אנונימי זה <div st...,4,0,0,0,0,...,1,0,1,0,15,15,78,0,5,10
3,0,1,גליה18,נשירה מאסיבית,"אינני יודעת אם זה הפורום המתאים, אבל לא מצאתי ...",2,0,0,0,0,...,2,0,1,0,75,75,79,0,2,73
4,0,1,Juliette,קבוצת תמיכה,אם אתם סובלים מדיכאון או מניה דיפרסיה ומעוניינ...,0,0,0,0,0,...,0,0,1,0,38,38,80,0,2,36


In [5]:
# save to column and remove stopwords
with open('he_stopwords.txt', encoding="utf8") as f:
    stopwords = f.read().splitlines()
print(f"Hebrew stopwords document is {type(stopwords)} type, {len(stopwords)} words length")

Hebrew stopwords document is <class 'list'> type, 499 words length


In [9]:
# create one column from two separate (emotional and informative)
file.loc[(file['emotional'] == 1) & (file['informative'] == 0), 'label'] = 1
file.loc[(file['emotional'] == 0) & (file['informative'] == 1), 'label'] = 0

# drop rows with both emotional and informative labels, because they are too unbalanced and are not important for the task
file = file[file['label'].notnull()]

# choose relevant columns
data = file[['post','title', 'Forum','label']]
data['label'].unique()

array([0., 1.])

In [10]:
# drop rows with nan values
data.dropna(inplace=True)

# check data
print(data.shape)
data.head()

(2101, 4)


C:\Users\olgaf\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,post,title,Forum,label
36,"מחפש פסיכיאטר טוב באזור המרכז(שעובד עם קופ""ח מ...",המלצה,1,0.0
37,חודשיים והוא מדכא לי את התאבון ומוריד אותי במש...,לוסטרל כדור נורא,1,0.0
38,מישהו/י שהיו בדיכאון והיה להם גם חרדה באותו הז...,שאלה..,1,0.0
39,לחברי הפורום שלום :) <br >איפה אפשר למצוא טיפו...,טיפול קובצתי מחפש,1,0.0
40,"שלום לכולם,<br >אני כיום בן 25<br >לפני כ3 שני...",חרדה,1,0.0


In [17]:
cleaner = Hebrew_preprocessing()
data['cleaned'] = data.loc[:,'post'].apply(lambda x: cleaner.regex_replacement(x))
data['tokens'] =  data.loc[:,'cleaned'].apply(lambda x: cleaner.tokenize(x))
data['punctuations'] =  data.loc[:,'cleaned'].apply(lambda x: cleaner.punctuation(x))
data['punctuation_count'] =  data.loc[:,'cleaned'].apply(lambda x: cleaner.punctuation_count(x))
data['numbers'] =  data.loc[:,'cleaned'].apply(lambda x: cleaner.numbers(x))
data['numbers_count'] =  data.loc[:,'cleaned'].apply(lambda x: cleaner.numbers_count(x))
data['stopwords_count'] = data.loc[:,'tokens'].apply(lambda x: cleaner.count_stopwords(x,stopwords))
data['stopwords_removed'] = data.loc[:,'tokens'].apply(lambda x: cleaner.remove_stopwords(x,stopwords))
data.shape

C:\Users\olgaf\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\olgaf\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\olgaf\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

(2101, 12)

In [15]:
data

,post,title,Forum,label,cleaned,tokens,punctuations,punctuation_count,numbers,numbers_count,stopwords_count,stopwords_removed
36,"מחפש פסיכיאטר טוב באזור המרכז(שעובד עם קופ""ח מ...",המלצה,1,0.0,"מחפש פסיכיאטר טוב באזור המרכז שעובד עם קופ""ח מ...","[מחפש, פסיכיאטר, באזור, המרכז, שעובד, קופ""ח, ש...",[.],1,[],0,7,"[מחפש, פסיכיאטר, באזור, המרכז, שעובד, קופ""ח, ש..."
37,חודשיים והוא מדכא לי את התאבון ומוריד אותי במש...,לוסטרל כדור נורא,1,0.0,חודשיים והוא מדכא לי את התאבון ומוריד אותי במש...,"[חודשיים, מדכא, התאבון, ומוריד, במשקל, שלד, מח...",[.],1,[],0,8,"[חודשיים, מדכא, התאבון, ומוריד, במשקל, שלד, מח..."
38,מישהו/י שהיו בדיכאון והיה להם גם חרדה באותו הז...,שאלה..,1,0.0,מישהו י שהיו בדיכאון והיה להם גם חרדה באותו הז...,"[שהיו, בדיכאון, והיה, חרדה, באותו, פניתם, לטיפ...","[,, ?, -, ?, ,, ?, ?, ?, ?, ., .]",11,[],0,20,"[שהיו, בדיכאון, והיה, חרדה, באותו, פניתם, לטיפ..."
39,לחברי הפורום שלום :) <br >איפה אפשר למצוא טיפו...,טיפול קובצתי מחפש,1,0.0,לחברי הפורום שלום איפה אפשר למצוא טיפול קבוצתי...,"[לחברי, למצוא, טיפול, קבוצתי, באזור, השרון, חד...","[?, .]",2,[],0,7,"[לחברי, למצוא, טיפול, קבוצתי, באזור, השרון, חד..."
40,"שלום לכולם,<br >אני כיום בן 25<br >לפני כ3 שני...",חרדה,1,0.0,"שלום לכולם, אני כיום בן 25 לפני כ3 שנים התחילו...","[כיום, התחילו, קשיים, בנשימה, ומין, שיעול, טור...","[,, ., ,, ,, ,, ', ., ., ., ., ., ?, ?]",13,"[25, 3]",2,57,"[כיום, התחילו, קשיים, בנשימה, ומין, שיעול, טור..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2677,"שלום רב,\nיש לי סכרת נעורים כבר 7 שנים. רוב הש...",בקשת עזרה,2,0.0,"שלום רב, יש לי סכרת נעורים כבר 7 שנים. רוב השנ...","[רב, סכרת, נעורים, השנים, מאוזנת, באזור, מפחית...","[,, ., ,, -, ., ., ., ,, ., ., ., ., ,, ., ,, ...",17,"[7, 12]",2,38,"[רב, סכרת, נעורים, השנים, מאוזנת, באזור, מפחית..."
2678,היי. אובחנתי השבוע עם אנורקסיה נרבוזה מהסוג המ...,אנורקסיה נרבוזה,2,1.0,היי. אובחנתי השבוע עם אנורקסיה נרבוזה מהסוג המ...,"[היי, אובחנתי, אנורקסיה, נרבוזה, מהסוג, המטהר,...","[., ., ., ., ., ., ., ., ., .]",10,[3],1,40,"[היי, אובחנתי, אנורקסיה, נרבוזה, מהסוג, המטהר,..."
2679,אספר על עצמי קצת אני בת 23 הפעם הראשונה היתה ל...,עזרהה,2,0.0,אספר על עצמי קצת אני בת 23 הפעם הראשונה היתה ל...,"[אספר, עצמי, הפעם, הראשונה, קורה, בחודש, מקיאה...","[,, ., ., ., ,, ., ?]",7,[23],1,59,"[אספר, עצמי, הפעם, הראשונה, קורה, בחודש, מקיאה..."
2680,כמה אפשר לקבל בממוצע? ללא אשפוזים אבל במרפאה ל...,הפרעות אכילה וביטוח לאומי,2,0.0,כמה אפשר לקבל בממוצע? ללא אשפוזים אבל במרפאה ל...,"[בממוצע, אשפוזים, במרפאה, להפרעות, אכילה, חולי...","[?, ., ., !]",4,[],0,10,"[בממוצע, אשפוזים, במרפאה, להפרעות, אכילה, חולי..."


In [ ]:
# class Hebrew_preprocessing:
#     def __init__ (self, additional_replacements = []):
     
#         #list of tuples, where tuples are paterns to be replaced
#         base_replacements = [
#                (r'(https.*$ )', ' '),           # remove links. here regex is used instead of Beautiful soup, because Beutiful soup doesn't work with Hebrew.
#                (r'href="(.*?)"', ' '),
#                (r'\\n?(https.*)\\n', ' '),
#                (r'(https.*)\\n', ' '),          # remove html tags
#                (r'(\&nbsp;)', ' '),             
#                (r'(\\&quot;)', ' '),
#                (r'<.*?>', ' '),                 # remove string in <> 
#                (r'\n',' '),                     # remove newline
#                (r'(\\r)',' '),
#                (r'[\:;&#=\{\}\(\)]',' '),       # remove special chatacters. again, [a-z0-9] won't work because of Hebrew
#                (r'(\/+)', ' '),
#                (r'(\\+)', ' '),
#                (r'(\++)', ' '),
#                (r' - ', r' '),
#                (r'(\s+)', r' ')                 # remove multiple spaces, created by the above cleaning 
#               ]
#         self.replacements = base_replacements + additional_replacements
        
#     def regex_replacement(self, string:str, custom_replacements:list[tuple[str,str]] = []) -> str:             
#         for pat,repl in self.replacements + custom_replacements:
#             string = re.sub(pat, repl, string)
#         return string        
    
#     def tokenize(self, text:str) -> tuple[list[str], list[str], list[str]]:
#         tokens = tokenizer.tokenize(text)
#         groups = defaultdict(list)
#         for grp, token, token_num, (start_index, end_index) in tokens:
#             group,tok = str(grp), str(token)
#             if group == 'Groups.HEBREW':
#                 groups['tokens'].append(token)
#             if group == 'Groups.PUNCTUATION':
#                 groups['punct'].append(token)
#             if group == 'Groups.NUMBER':
#                 groups['numbers'].append(token)
#         return groups['tokens'], groups['punct'], groups['numbers']
    
#     """
#     Set up Hebrew Lematizer before the function. It need Go pre-installed. 
#     Documentation: https://nlp.biu.ac.il/~rtsarfaty/onlp/hebrew/documentation
#     IP of YAP server: ip = '127.0.0.1:8000'
#     yap = YapApi()
#     """
#     def lemmatize(self, list_tokens:list[str]) -> str:
#         text = ' '.join(list_tokens)
#         tokenized_text, segmented_text, lemmas, dep_tree, md_lattice, ma_lattice = yap.run(text, ip)
#         return lemmas
#     """
#     Hebrew stopwords are loaded from "he_stopwords.txt" file. 
#     """
#     def count_stopwords(self,text:str, stopwords):
#         tokens = text.split(' ')   
#         stopw = []
#         count = 0
#         for w in stopwords:
#             if w in tokens: 
#                 for i in range(len(tokens)): 
#                     if (w == tokens[i]): 
#                         count = count + 1
#                 stopw.append(w)
#         return count
    
#     def remove_stopwords(self,text:str, stopwords):
#         stopw = []
#         for w in stopwords:
#             while w in line:
#                 line.remove(w)
#         return line

#     def save_csv(self, dataFrame, name:str):
#         dataFrame.to_csv(name+'.csv',index=False)
        